# Tech Challenge 4 - Predição de Obesidade

**Objetivo:** Desenvolver modelos de Machine Learning para predizer níveis de obesidade com pelo menos 75% de acurácia.

**Dataset:** Dados de obesidade com features pré-processadas e target 'obesidade' (0-6 níveis).

In [8]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("Bibliotecas carregadas com sucesso!")

Bibliotecas carregadas com sucesso!


In [9]:
# Carregamento e análise inicial dos dados
df = pd.read_csv('../DATASETS/dados_machine_learning.csv', index_col=0)

print(f"Shape do dataset: {df.shape}")
print(f"\nColunas disponíveis: {list(df.columns)}")
print(f"\nTarget variable 'obesidade' - distribuição:")
print(df['obesidade'].value_counts().sort_index())
print(f"\nInformações do dataset:")
print(df.info())

Shape do dataset: (2111, 30)

Colunas disponíveis: ['IMC_tratado', 'idade_scaler', 'altura_scaler', 'peso_scaler', 'IMC_scaler', 'IMC_tratado_scaler', 'idade_min_max_scaler', 'altura_min_max_scaler', 'peso_min_max_scaler', 'IMC_min_max_scaler', 'IMC_tratado_min_max_scaler', 'genero', 'idade', 'altura', 'peso', 'historico_familiar', 'frequencia_consumo_alimentos_caloricos', 'frequencia_consumo_vegetais', 'numero_refeicoes', 'consumo_lanches_entre_refeicoes', 'fuma', 'CH2O', 'monitoramento_calorias', 'frequencia_atividade_fisica', 'tempo_diario_uso_dispositivos_eletronicos', 'consumo_alcool', 'tipo_transporte', 'IMC', 'obesidade_agrupada', 'obesidade']

Target variable 'obesidade' - distribuição:
obesidade
0    272
1    287
2    290
3    290
4    351
5    297
6    324
Name: count, dtype: int64

Informações do dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 2111 entries, 0 to 2110
Data columns (total 30 columns):
 #   Column                                     Non-Null Count  Dtype 

In [10]:
# Criação da coluna target obesidade_binary
df['obesidade_binary'] = df['obesidade'].apply(lambda x: 0 if x <= 3 else 1)

# REMOVENDO FEATURES COM DATA LEAKAGE (IMC, peso, altura)
features_basicas = ['genero', 'idade', 'historico_familiar', 'frequencia_consumo_alimentos_caloricos',
                   'frequencia_consumo_vegetais', 'numero_refeicoes', 'consumo_lanches_entre_refeicoes',
                   'fuma', 'CH2O', 'monitoramento_calorias', 'frequencia_atividade_fisica',
                   'tempo_diario_uso_dispositivos_eletronicos', 'consumo_alcool', 'tipo_transporte']

X = df[features_basicas]
y = df['obesidade_binary']

print(f"Features shape: {X.shape}")
print(f"Target distribution: {y.value_counts()}")

# Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalização
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Features shape: (2111, 14)
Target distribution: obesidade_binary
0    1139
1     972
Name: count, dtype: int64


In [11]:
# Modelo 1: Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(classification_report(y_test, rf_pred))

Random Forest Accuracy: 0.9125
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       228
           1       0.92      0.89      0.90       195

    accuracy                           0.91       423
   macro avg       0.91      0.91      0.91       423
weighted avg       0.91      0.91      0.91       423



In [12]:
# Modelo 2: Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_pred = gb_model.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f"Gradient Boosting Accuracy: {gb_accuracy:.4f}")
print(classification_report(y_test, gb_pred))

Gradient Boosting Accuracy: 0.8865
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       228
           1       0.86      0.90      0.88       195

    accuracy                           0.89       423
   macro avg       0.89      0.89      0.89       423
weighted avg       0.89      0.89      0.89       423



In [ ]:
# Modelo 3: SVM
svm_model = SVC(kernel='rbf', random_state=42)
svm_model.fit(X_train_scaled, y_train)
svm_pred = svm_model.predict(X_test_scaled)
svm_accuracy = accuracy_score(y_test, svm_pred)

print(f"SVM Accuracy: {svm_accuracy:.4f}")
print(classification_report(y_test, svm_pred))

In [ ]:
# Análise detalhada das métricas
models = {
    'Random Forest': (rf_pred, rf_accuracy),
    'Gradient Boosting': (gb_pred, gb_accuracy),
    'SVM': (svm_pred, svm_accuracy)
}

print("=== ANÁLISE DETALHADA DAS MÉTRICAS ===")
for name, (pred, acc) in models.items():
    print(f"\n{name}:")
    print(f"Accuracy: {acc:.4f}")
    print(classification_report(y_test, pred, target_names=['Não Obeso', 'Obeso']))

In [ ]:
# Matrizes de Confusão
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (name, (pred, _)) in enumerate(models.items()):
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Não Obeso', 'Obeso'],
                yticklabels=['Não Obeso', 'Obeso'],
                ax=axes[i])
    axes[i].set_title(f'{name}')
    axes[i].set_xlabel('Predito')
    axes[i].set_ylabel('Real')

plt.tight_layout()
plt.show()

In [ ]:
# Comparação final
results = {
    'Random Forest': rf_accuracy,
    'Gradient Boosting': gb_accuracy,
    'SVM': svm_accuracy
}

print("=== COMPARAÇÃO FINAL ===")
for model, acc in results.items():
    print(f"{model}: {acc:.4f}")

best_model = max(results, key=results.get)
print(f"\nMelhor modelo: {best_model} ({results[best_model]:.4f})")

target_accuracy = 0.75
models_above_target = [model for model, acc in results.items() if acc >= target_accuracy]

if models_above_target:
    print(f"\nModelos ≥75%: {models_above_target}")
else:
    print(f"\nNenhum modelo atingiu 75% de acurácia")